In [1]:
# Install PyMuPDF and pdfrw
# !pip install pymupdf pdfrw PyPDF2

In [15]:
import fitz  # PyMuPDF
from pdfrw import PdfReader, PdfWriter, IndirectPdfDict, PdfName, PdfObject, PdfDict

# Function to extract form field information from a PDF
def extract_form_fields(pdf_path):
    pdf = fitz.open(pdf_path)
    form_fields = []

    for page in pdf:
        widgets = page.widgets()  # Get the form widgets (form fields)
        if widgets:  # If form fields exist on this page
            for widget in widgets:
                field_type = widget.field_type  # Field type (checkbox, text, etc.)
                field_name = widget.field_name  # Field name
                try:
                    field_value = widget.get_text("text")  # Field value
                except:
                    field_value = None
                form_fields.append({
                    "name": field_name,
                    "type": field_type,
                    "value": field_value,
                })

    pdf.close()
    return form_fields

# Extract the form fields from your PDF
pdf_path = "data/sample.pdf"
fields = extract_form_fields(pdf_path)

# Print the form fields to understand their structure
for field in fields:
    print(f"Field Name: {field['name']}, Field Type: {field['type']}, Field Value: {field['value']}")


Field Name: Text1, Field Type: 7, Field Value: None
Field Name: Text2, Field Type: 7, Field Value: None
Field Name: Text3, Field Type: 7, Field Value: None
Field Name: Text4, Field Type: 7, Field Value: None
Field Name: Text5, Field Type: 7, Field Value: None
Field Name: Text6, Field Type: 7, Field Value: None
Field Name: Text7, Field Type: 7, Field Value: None
Field Name: Text10, Field Type: 7, Field Value: None
Field Name: Text12, Field Type: 7, Field Value: None
Field Name: Text20, Field Type: 7, Field Value: None
Field Name: Check Box5, Field Type: 2, Field Value: None
Field Name: Check Box6, Field Type: 2, Field Value: None
Field Name: Check Box7, Field Type: 2, Field Value: None
Field Name: Check Box8, Field Type: 2, Field Value: None
Field Name: Button1, Field Type: 1, Field Value: None
Field Name: Button2, Field Type: 1, Field Value: None
Field Name: Check7, Field Type: 2, Field Value: None
Field Name: Check 8, Field Type: 2, Field Value: None
Field Name: Text8, Field Type: 7, 

In [21]:
# Create a dictionary with field names and their values to fill
from PyPDF2.generic import NameObject
field_values = {
    "(Check Box10)": "/Yes",  # Example checkbox (use "Yes" or "No" to check/uncheck)
    "(Check Box11)": "/Yes",  # Example text field
}

# Function to fill form fields in a PDF
def fill_form_fields(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        annotations = page['/Annots']
        # print(annotations)
        for annotation in annotations:
            if annotation['/Subtype'] == '/Widget':  # It's a form field
                field_name = annotation.get('/T')  # Field name
                # print(field_name)
                # if "Check" in field_name:
                    # print(annotation['/FT'])
                # if field_name in field_values:
                # print(annotation.keys())
                # if (annotation['/FT'] == '/Tx'):  # Checkbox
                #     print("Annotation:", annotation)
                #     annotation["/V"] = "Hola"
                if annotation.get(PdfName.FT) == PdfName.Tx:
                    # Properly set the field value
                    annotation[PdfName.V] = "Raphita comeme los huevos" #PdfObject()

                if annotation.get(PdfName.FT) == PdfName.Btn:
                        print("Annotation:", annotation)
                        print(annotation.get(PdfName.T))
                        print("\tV:",annotation.get(PdfName.V))
                        print("\tAS:",annotation.get(PdfName.AS))
                        print("\tAP:",annotation.get(PdfName.AP))
                        print("\tFf:",annotation.get(PdfName.Ff))


                        # Properly set the checkbox value
                        annotation[PdfName.Ff] = 0
                        annotation[PdfName.V] = NameObject(PdfName.Yes) 
                        annotation[PdfName.AS] = NameObject(PdfName.Yes)
                        print("New Annotation:", annotation)
                        print("\tV:",annotation.get(PdfName.V))
                        print("\tAS:",annotation.get(PdfName.AS))
                        print("\tAP:",annotation.get(PdfName.AP))
                        print("\n\n")
                    # writer_annot = annotation.getObject()
                    # annotation.update({
                    #             NameObject("/V"): NameObject(PdfName.On),
                    #             NameObject("/AS"): NameObject(PdfName.On)
                    #         })



    # Write to the output PDF
    PdfWriter().write(output_pdf_path, trailer=pdf)
    

# Fill the form fields with specified values
input_pdf = "data/sample.pdf"
output_pdf = "data/sample_filled.pdf"

fill_form_fields(input_pdf, output_pdf, field_values)

print("Filled form has been created:", output_pdf)


Annotation: {'/AP': {'/D': {'/Off': (18, 0), '/Yes': (19, 0)}, '/N': {'/Off': (20, 0), '/Yes': (21, 0)}}, '/AS': '/Off', '/BS': {'/W': '0.5'}, '/DA': '(0 g /ZaDb 0 Tf)', '/F': '4', '/FT': '/Btn', '/M': "(D:20190905141541+02'00')", '/MK': {'/BC': ['0.753', '0.753', '0.753'], '/BG': ['1', '1', '1'], '/CA': '(8)'}, '/P': (109, 0), '/Rect': ['319.209076', '128.952759', '333.382294', '143.125992'], '/Subtype': '/Widget', '/T': '(Check Box5)', '/Type': '/Annot'}
(Check Box5)
	V: None
	AS: /Off
	AP: {'/D': {'/Off': (18, 0), '/Yes': (19, 0)}, '/N': {'/Off': (20, 0), '/Yes': (21, 0)}}
	Ff: None
New Annotation: {'/AP': {'/D': {'/Off': (18, 0), '/Yes': (19, 0)}, '/N': {'/Off': (20, 0), '/Yes': (21, 0)}}, '/AS': '/Yes', '/BS': {'/W': '0.5'}, '/DA': '(0 g /ZaDb 0 Tf)', '/F': '4', '/FT': '/Btn', '/M': "(D:20190905141541+02'00')", '/MK': {'/BC': ['0.753', '0.753', '0.753'], '/BG': ['1', '1', '1'], '/CA': '(8)'}, '/P': (109, 0), '/Rect': ['319.209076', '128.952759', '333.382294', '143.125992'], '/Subt

In [104]:
from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject

# Function to fill checkboxes in a PDF
def fill_checkboxes(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        if '/Annots' not in page:
            continue  # No annotations on this page

        annotations = page['/Annots']
        for annotation in annotations:
            if annotation.get(PdfName.Subtype) == PdfName.Widget:
                field_name = annotation.get(PdfName.T)  # Field name
                if field_name and field_name in field_values:
                    field_value = field_values[field_name]

                    # If it's a checkbox (button type), set it accordingly
                    if annotation.get(PdfName.FT) == PdfName.Btn:
                        # Check if it's a checkbox with multiple appearances
                        # Find the default value key in the dictionary
                        normal_appearance = annotation.get(PdfName.AP, {}).get(PdfName.N, {})
                        
                        # Check if the desired value matches an existing key in the appearance
                        if field_value == 'checked':
                            if PdfName.On in normal_appearance:
                                annotation[PdfName.V] = 
                                annotation[PdfName.AS] = PdfName.On
                        else:  # 'unchecked'
                            annotation[PdfName.V] = PdfName.Off
                            annotation[PdfName.AS] = PdfName.Off

    # Write the updated PDF
    PdfWriter(output_pdf_path, trailer=pdf).write()

# Define the checkbox values to set
field_values = {
    '(Check Box5)': 'checked',  # This checkbox should be checked
    '(Check Box6)': 'unchecked',  # This checkbox should be unchecked
}

input_pdf = "data/sample.pdf"
output_pdf = "data/sample_filled2.pdf"

# Fill the checkboxes in the PDF
fill_checkboxes(input_pdf, output_pdf, field_values)

print("Filled PDF created:", output_pdf)


TypeError: 'dict' object is not callable